# ROI - Build Model

## 0. Imports


In [ ]:
import sys
# voeg hier de locatie van de ROI code toe:
sys.path.append("D:\Work\git\GIS_tools\Code")

from data_structures.dhydro_data import DHydroData

## 1. Input

Hier kan je uitleg toeveogen Ludo. Dubbelklik op deze cell, pas tekst aan, run cell


In [ ]:
# voeg hier de locatie van de input data toe:
folder = r"D:\Work\Project\P1414"

# voeg hier de locatie van de output data toe:
output_folder = r"D:\Work\Project\P1414\Models\Combined\V16_HHD"

# selecteer hier welke acties je wil uitvoeren:
build_database = True
load_gpkgs = False
build_model = True


gpkg_file = folder + r"\GIS\HYDAMO\Combined_test_v16_HHD.gpkg"
gpkgs_list = [
    r"D:\Work\Project\P1414\GIS\HYDAMO\HHSK.gpkg",
    r"D:\Work\Project\P1414\GIS\HYDAMO\HDSR.gpkg",
    r"D:\Work\Project\P1414\GIS\HYDAMO\HHD.gpkg",
    r"D:\Work\Project\P1414\GIS\HYDAMO\HHR.gpkg",
    r"D:\Work\Project\P1414\GIS\HYDAMO\WAGV.gpkg",
    r"D:\Work\Project\P1414\GIS\HYDAMO\ARKNZK.gpkg",
    r"D:\Work\Project\P1414\GIS\HYDAMO\Rijntakken.gpkg",
    r"D:\Work\Project\P1414\GIS\HYDAMO\RMM_closed.gpkg",
    r"D:\Work\Project\P1414\GIS\HYDAMO\noordzee_hoog.gpkg",
    r"D:\Work\Project\P1414\GIS\HYDAMO\markermeer.gpkg",
    r"D:\Work\Project\P1414\GIS\HYDAMO\tunnels.gpkg",
    r"D:\Work\Project\P1414\GIS\HYDAMO\Ontbrekende_stuwen.gpkg",
]

config_dhydro = r"combined_HHD_config"
config_list = [
    r"hhsk_config",
    r"hdsr_config",
    r"hhd_config",
    r"hhr_config",
    r"wagv_config",
    r"ark_nzk_config",
    r"rijntakken_config",
    r"rijnmaasmonding_config",
    r"noordzee_config",
    r"markermeer_config",
]
snap_dist_list = [0, 0, 10, 10, 50, 10, 10, 100, 100, 100, 0, 0]

defaults = r"defaults"

## 2. Build geopackage database

Hier kan je uitleg toeveogen Ludo. Dubbelklik op deze cell, pas tekst aan, run cell


In [ ]:
if build_database:
    dhd = DHydroData()
    for ix, config in enumerate(config_list):
        print("\n" + config)

        dhd.hydamo_from_raw_data(
            defaults=defaults, config=config, branch_snap_dist=snap_dist_list[ix]
        )
        try:
            dhd.fixed_weirs_from_raw_data(config=config, defaults=defaults)
        except AttributeError:
            pass

    dhd.clip_structures_by_branches()
    dhd.fixed_weirs_from_raw_data(config="wegen_config", defaults=defaults)
    dhd.fixed_weirs_from_raw_data(config="relief_config", defaults=defaults)
    dhd.fixed_weirs_from_raw_data(config="noordzeekeringen_config", defaults=defaults)
    # dhd.dambreaks_from_config(config="dambreak_v0_config", defaults=defaults)
    dhd.hydamo_to_gpkg(output_gpkg=gpkg_file)



## 3. Load geopackage

Hier kan je uitleg toeveogen Ludo. Dubbelklik op deze cell, pas tekst aan, run cell


In [ ]:
if load_gpkgs:
    dhd = DHydroData()
    for ix, gpkg in enumerate(gpkgs_list):
        print("\n" + gpkg)

        # 2. load data
        dhd.hydamo_from_gpkg(gpkg, branch_snap_dist=snap_dist_list[ix])

    dhd.fixed_weirs_from_raw_data(config="wegen_config", defaults=defaults)
    dhd.fixed_weirs_from_raw_data(config="relief_config", defaults=defaults)
    dhd.fixed_weirs_from_raw_data(config="noordzeekeringen_config", defaults=defaults)
    # dhd.dambreaks_from_config(config="dambreak_v0_config", defaults=defaults)
    dhd.hydamo_to_gpkg(output_gpkg=gpkg_file)


## 4. Build model

Hier kan je uitleg toeveogen Ludo. Dubbelklik op deze cell, pas tekst aan, run cell


In [ ]:
if build_model:
    # 1. initialize an instance of DHydamoData
    dhd = DHydroData()

    # 2. load data
    dhd.hydamo_from_gpkg(gpkg_file)

    # remove brug as it needs a cs
    del dhd.ddm.brug
    dhd.features.remove("brug")
    dhd.ddm.pomp["maximalecapaciteit"] = 0

    # 3. save as dhydro model
    dhd.to_dhydro(config=config_dhydro, output_folder=output_folder)
